# Multi-LLM Routing

## Imports

In [3]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from anthropic import Anthropic
from IPython.display import Markdown, display

In [4]:
load_dotenv(override=True)

True

In [6]:
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set (and this is optional)")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")

if deepseek_api_key:
    print(f"DeepSeek API Key exists and begins {deepseek_api_key[:3]}")
else:
    print("DeepSeek API Key not set (and this is optional)")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AI
DeepSeek API Key exists and begins sk-


In [ ]:
question = ""
question += ""
messages = [{"role": "user", "content": question}]

In [ ]:
def gpt_model():
    openai = OpenAI()

    response = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages = messages,
)
    answer = response.choices[0].message.content
    return display(Markdown(answer))

In [ ]:
def anthropic_model():
    claude = Anthropic()

    model_name = "claude-3-7-sonnet-latest"
    response = claude.messages.create(model=model_name, messages=messages, max_tokens=1000)
    answer = response.content[0].text
    return display(Markdown(answer))

In [ ]:
def deepseek_model():
    deepseek = OpenAI(api_key=deepseek_api_key, base_url="https://api.deepseek.com/v1")
    model_name = "deepseek-chat"

    response = deepseek.chat.completions.create(model=model_name, messages=messages)
    answer = response.choices[0].message.content

    return display(Markdown(answer))


In [ ]:
def google_model():
    gemini = OpenAI(api_key=google_api_key, base_url="https://generativelanguage.googleapis.com/v1beta/openai/")
    model_name = "gemini-2.0-flash"

    response = gemini.chat.completions.create(model=model_name, messages=messages)
    answer = response.choices[0].message.content

    return display(Markdown(answer))

In [ ]:
!ollama pull llama3.2

def llama_model():
    ollama = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
    model_name = "llama3.2"

    response = ollama.chat.completions.create(model=model_name, messages=messages)
    answer = response.choices[0].message.content

    return display(Markdown(answer))

In [ ]:
provide_to_llm = {
    'OpenAI': gpt_model,
    'Anthropic': anthropic_model,
    'DeepSeek': deepseek_model,
    'Gemini': google_model,
    'Llama': llama_model
}

In [ ]:
def get_llm(question):
    user_prompt = (
        'Given the following AWS related question, which of these LLM providers would do best in answering?'
        'OpenAI, Anthropic, DeepSeek, Gemini, Llama\n'
        'Just reply only with the LLM Model name\n'
        f'Question: {question}'
    )
    response = openai.ChatCompletion.create(
        model = 'gpt-3.5-turo',
        messages = [{'role': 'user', 'content': user_prompt}],
        max_tokens = 2
    )
    return response.choice[0].message.content


In [ ]:
def aws_router(question):
    print(f'User question: {question}')
    llm_model = get_llm(question)
    print(f'Router LLM chose the following mode: {llm_model}')
    llm_func = provide_to_llm.get(llm_model, lambda q: 'Sorry unsupported LLM or error.')
    answer = llm_func(question)
    print(f'Answer from {llm_model}: {answer}')